In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import jit

## Regular Gillespie (without Numba)

In [ ]:
# a function that performs Gillespie's direct algorithm but with regular time steps
def gillespie_regular(a0,l0,m0,tf,tsteps,tchange,cnew,
                      delta_f=-2,taul=0.01,Ki=18,Ka=2900,N=6,kR=0.1,kB=0.2,mMax=24,kAct=30):
    '''
    Returns an array of the activity, number of ligands bound and methylation level
    for a rate-based MWC model including (de)activation, ligand (un)binding and
    (de)methylation, implemented using Gillespie's direct algorithm
    
    Parameters
    ----------
    a0: float
        initial activity
    l0: float
        initial number of ligands bound
    m0: float
        initial methylation level
    tf: float
        final time of simulation in sec
    tsteps: int
        number of evenly-spaced time steps
    tchange: float
        time in sec at which the concentration is changed
    cnew: float
        new concentration in micromolar
    delta_f: float
        energy penalty per methylated site in units of kT
    taul: float
        ligand (un)binding time scale
    Ki: float
        dissocation constant for inactive cluster in micromolar
    Ka: float
        dissociation constant for active cluster in micromolar
    N: int
        number of receptors per cluster
    kR: float
        methylation rate per sec
    kB: float
        demethylation rate per sec
    mMax: int
        number of methylation sites (note: mMax = 4*N)
    kAct: float
        activation rate per sec
        
    Returns
    -------
    av_states: NumPy array
        array of activity, number of ligands bound and methylation level, averaged between
        the time steps given in time_regular
    '''
    # set the initial state
    state = np.array([a0,l0,m0])
    # set the regular time steps
    time_regular = np.linspace(0,tf,tsteps)
    
    # where all the averaged results are stored
    av_states = np.array([state])
    
    time = 0
    
    # baseline concentration
    c = 100
    
    # execute the actual algorithm
    for i in range(tsteps):
        # store all the results
        all_states = np.array([state])
    
        while time < time_regular[i]:
            # change concentration after tincrease
            if time > tchange:
                c = cnew
                
            # calculate free-energy difference
            f = N*np.log((1 + c/Ki)/(1 + c/Ka)) + delta_f*(state[2] - N/2)
            
            # set rate constants for (de)activation
            ############## is the sign correct here?
            kDeact = kAct*np.exp(f)

            # set the events with the corresponding rates
            rates = np.zeros(4)
            events = np.zeros((4,3))

            # check methylation states
            meth_ceiling = state[2] < mMax
            meth_floor = state[2] > 0

            # for inactive cluster
            if state[0] == 0:
                # calculate rate of binding
                kplus = 1/(taul*(c + Ki))
                
                # ligand binding
                rates[0] = kplus*c*(N - state[1])
                events[0] = np.array([0,1,0])
                
                # ligand unbinding
                rates[1] = kplus*Ki*state[1]
                events[1] = np.array([0,-1,0])

                # methylation; only possible if a site is available
                rates[2] = kR*meth_ceiling
                events[2] = np.array([0,0,1])

                # activity switching
                rates[3] = kAct
                events[3] = np.array([1,0,0])

            # for active cluster
            else:
                # calculate rate of binding
                kplus = 1/(taul*(c + Ka))
                
                # ligand binding
                rates[0] = kplus*c*(N - state[1])
                events[0] = np.array([0,1,0])
                
                # ligand unbinding
                rates[1] = kplus*Ka*state[1]
                events[1] = np.array([0,-1,0])

                # demethylation; only possible if a site is occupied
                rates[2] = kB*meth_floor
                events[2] = np.array([0,0,-1])

                # deactivation
                rates[3] = kDeact
                events[3] = np.array([-1,0,0])

            # this samples the time until the next event
            u = np.random.uniform(0,1)
            dt = -np.log(u)/np.sum(rates)

            # update time
            time += dt

            # determine which event occurs
            p = np.random.uniform(0,1)*np.sum(rates)
            mask = p < np.cumsum(rates)
            event_idx = np.min(np.where(mask == True))

            # perform the event
            state = state + events[event_idx]
            all_states = np.append(all_states,[state],axis=0)
        
        # average between time points
        av_states = np.append(av_states,[np.mean(all_states,axis=0)],axis=0)
        
    return av_states[1:,:]

In [ ]:
# set up an objective function to use for lmfit

def lmf_lsq_resid(params,xdata,ydata,yerrs,model,output_resid=True):
    if output_resid == True:
        for i, xval in enumerate(xdata):
            if i == 0:
                residual = (ydata[i] - model(xdata[i],params))/yerrs[i]
            else:
                residual = np.append(resid,(ydata[i] - model(xdata[i],params))/yerrs[i])
        return residual
    else:
        ymodel = []
        for i, xval in enumerate(xdata):
            ymodel.append(model(xdata[i],params))
        return ymodel
    
def model_fitting(model,xdata,ydata,yerrs,params):
    output_resid = True
    
    # this defines the minimiser object
    set_function = Minimizer(lmf_lsq_resid, params, fcn_args=(xdata, ydata, yerrs, model, output_resid), nan_policy='omit')
    
    # this perform least-square fitting
    result = set_function.minimize(method='leastsq')
    
    # this reports the results
    report_fit(results)
    
    # this obtains the model values
    model_val = lmf_lsq_resid(result.params,xdata,ydata,yerrs,model,output_resid=False)
    
    return model_val

In [ ]:
# perform the fitting

def heaviside(t):
    return t > 0

# define a parameters object
params = Parameters()
params.add_many(('G',0.12),('tau1',0.1),('tau2',10))

# define the actual model
def mattingly_model(t_val,params):
    value = params.valuesdict()
    return value['G']*np.exp(-t_val/value['tau2'])*(1 - np.exp(-t_val/value['tau1']))*heaviside(t_val)
                                                    
model = mattingly_model
xdata = [time]
ydata = [np.mean(test[:,:,0],axis=0)]
yerrs = [np.std(test[:,:,0],axis=0)]

model_val = model_fitting(model,xdata,ydata,yerrs,params)